In [ ]:
from datasets import load_dataset
import numpy as np

In [ ]:
raw_datasets = load_dataset("glue", "sst2")
raw_datasets

In [ ]:
dir(raw_datasets['train'])

In [ ]:
type(raw_datasets['train'])

In [ ]:
raw_datasets['train'].data

In [ ]:
raw_datasets['train'][0]

In [ ]:
raw_datasets['train'][50000:50003]

In [ ]:
raw_datasets['train'].features

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenized_sentences = tokenizer(raw_datasets['train'][0:3]['sentence'])
from pprint import pprint
pprint(tokenized_sentences)

In [ ]:
def tokenize_fn(batch):
    return tokenizer(batch['sentence'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'my_trainer',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1,
)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2)
model

In [ ]:
from torchinfo import summary
#summary(model)
summary(model, input_size=(16,512), dtypes=['torch.IntTensor'], device='cpu')

In [ ]:
# Just for a sanity check
params_before = []
for name, p in model.named_parameters():
    params_before.append(p.detach().cpu().numpy())
len(params_before)

In [ ]:
from transformers import Trainer
from datasets import load_metric

In [ ]:
metric = load_metric('glue', 'sst2')
metric.compute(predictions=[1, 0, 1], references=[1, 0, 0])

In [ ]:
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('my_saved_model')

In [ ]:
from transformers import pipeline
newmodel = pipeline('sentiment-analysis', model='my_saved_model', device=0)

In [ ]:
newmodel('This movie is great')

In [ ]:
newmodel('This movie could be great but they did not make it')

In [ ]:
import json
config_path = 'my_saved_model/config.json'
with open (config_path, 'r') as f:
    config = json.load(f)
config

In [ ]:
config['id2label'] = {0: 'NEGATIVE', 1: 'POSITIVE'}
with open (config_path, 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
!cat my_saved_model/config.json

In [ ]:
newmodel = pipeline('text-classification', model='my_saved_model', device=0)

In [ ]:
newmodel('This movie is great!')

In [ ]:
params_after = []
for name, p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())
  

In [ ]:
for p1, p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1 - p2)))